In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
import keras_tuner as kt
from tensorflow import keras
from tensorflow.keras import layers

In [3]:
final_df = pd.read_csv("../data/final_features_with_targets.csv", parse_dates=['Date'], index_col='Date')

feature_cols = [col for col in final_df.columns if not col.endswith('_target_5d')]
target_cols = [col for col in final_df.columns if col.endswith('_target_5d')]

X = final_df[feature_cols]
y = final_df[target_cols]

FileNotFoundError: [Errno 2] No such file or directory: '../data/final_features_with_targets.csv'

In [ ]:
asset_main = "EEM"

asset_features = [col for col in X.columns if col.startswith(asset_main)]
X_asset = X[asset_features]
y_asset = y[f"{asset_main}_target_5d"]

data = pd.concat([X_asset, y_asset], axis=1).dropna()
X_asset = data[asset_features]
y_asset = data[f"{asset_main}_target_5d"]

# 80-20 train validation split 
split_index = int(len(X_asset) * 0.8)
X_train, X_val = X_asset.iloc[:split_index], X_asset.iloc[split_index:]
y_train, y_val = y_asset.iloc[:split_index], y_asset.iloc[split_index:]

#MLP Sequential model picked 
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dropout(0.1),
    Dense(1)  
])

model.compile(optimizer='adam', loss='mse')

early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=100,
    batch_size=32,
    callbacks=[early_stop],
    verbose=1
)

y_pred = model.predict(X_val).flatten()

rmse = np.sqrt(mean_squared_error(y_val, y_pred))
mae = mean_absolute_error(y_val, y_pred)
r2 = r2_score(y_val, y_pred)

print(f"\n MLP Evaluation for {asset_main}:")
print(f"RMSE: {rmse:.5f}")
print(f"MAE : {mae:.5f}")
print(f"R²  : {r2:.5f}")


In [ ]:

plt.figure(figsize=(12, 4))
plt.plot(y_val.index, y_val.values, label='Actual', color='black')
plt.plot(y_val.index, y_pred, label='Predicted', color='red')
plt.title(f'{asset_main} - Actual vs Predicted 5-day Returns')
plt.xlabel('Date')
plt.ylabel('5-day Return')
plt.legend()
plt.tight_layout()
plt.show()